# 目的
CryptoCompareのAPI[[1](#ref01)]でBTCJPYを取得するテスト。    
また、CryptoWatch[[2](#ref02)]とpybitflyer[[3](#ref03)]も使ってみる。   
（CryptoWatchは試さないかも）

## タイムスタンプについて
biyFlyerのタイムスタンプはunix timestampの様子[[4](#ref04)]。   
Pythonのdatetimeでも利用できる。たとえば[[5](#ref05)]を参照。

## 参考
* <a name="ref01">[1]</a>[bitFlyerの過去チャートを1時間足で取得する](http://blog.shoby.jp/entry/2017/06/14/005343)
* <a name="ref02">[2]</a>[bitFlyerの過去チャートを取得するAPI](http://blog.shoby.jp/entry/2017/05/24/093733)
* <a name="ref03">[3]</a>[bitFlyer LightningのAPIをPythonから使えるパッケージ「pybitflyer」を作りました](http://wolfin.hatenablog.com/entry/2016/08/29/010112)
* <a name="ref04">[4]</a>[PythonでbitFlyerのBTCFXのローソク足を取得してみた](http://www.bitbityen.com/entry/2017/11/22/080000http://www.bitbityen.com/entry/2017/11/22/080000)
* <a name="ref05">[5]</a>[Python 3 で少しだけ便利になった datetime の新機能](https://qiita.com/methane/items/d7ac3c9af5a2c659bc51)
* <a name="ref06">[6]</a>[CryptoCompare API for Developers](https://min-api.cryptocompare.com/)
* <a name="ref07">[7]</a>[yagays/pybitflyer: Python wrapper for bitFlyer's REST API.](https://github.com/yagays/pybitflyer)

# CryptoCompare
## 使い方
ヒストリカルデータを取得したい場合は、`histoAAAA`とパラメータを入れる。たとえば時間足が欲しい場合は次のようになる。
```
https://min-api.cryptocompare.com/data/histohour?fsym=BTC&tsym=JPY&limit=2000&e=bitFlyer&toTs=1446267600
```
分足（histominute）は、一週間分しか保存されないようである。   
パラメータと説明は[[6](#ref06)]を参照。   

## 利用上の注意
```
Our API is under a Creative Commons - Attribution Non-Commercial license. This means that it’s free as long as you attribute it to us and you are not making money from the data. If you are selling a product that uses our data we charge a fee. The idea is that if you start making a lot of money using our data, we would like something in return. If you are just providing a free service, you can use our data freely. Please make sure you credit us with a link if you use our data on your website or app.
```
つまり、取得したデータを用いてお金稼ぎをする場合はいくらかお金をもらうよ、ということ。

In [41]:
import urllib.request
import urllib.parse
import json

def getRateViaCrypto(histoticks, params):
    """getRateViaCrypto(params)
    get rate via CryptoCompare (https://min-api.cryptocompare.com/).
    params should be a dict object with items of str.
    <params>
    histoticks: day, hour, minute or some tick (see the site)
    params: parameters to send the site 
        fsym : currency symbol of interest (required)
        tsym : currency symbol to convert into (required)
        limit: limit of retrieved data (max: 2000)
        e    : the exchange to obtain data from (CCCAGG - by default)
        toTs : last unix timestamp to return data for
    </params>
    
    <return>
    retrieved data (json object)
    </return>
    """
    
    url = "https://min-api.cryptocompare.com/data/"
    url += "histo{}".format(histoticks) + "?" + urllib.parse.urlencode(params) 
    res = urllib.request.urlopen(url)
    result = json.loads(res.read().decode('utf-8'))
    return result

if __name__ == "__main__":
    histoticks = "hour"
    params = {
        "fsym": "BTC",
        "tsym": "JPY",
        "limit": "10",
        "e": "bitFlyer"
    }
    result = getRateViaCrypto(histoticks, params)
    for key in result.keys():
        if key != "Data":
            print(key, result[key])
        else:
            print(key)
            for data in result[key]:
                print(data)

Response Success
Type 100
Aggregated False
Data
{'time': 1525503600, 'close': 1065715, 'high': 1066200, 'low': 1064000, 'open': 1064147, 'volumefrom': 161.48, 'volumeto': 172006995.43}
{'time': 1525507200, 'close': 1068000, 'high': 1069519, 'low': 1065011, 'open': 1065715, 'volumefrom': 331.89, 'volumeto': 354447602.02}
{'time': 1525510800, 'close': 1069541, 'high': 1071000, 'low': 1066002, 'open': 1068000, 'volumefrom': 380.98, 'volumeto': 407261077.63}
{'time': 1525514400, 'close': 1075997, 'high': 1080000, 'low': 1068730, 'open': 1069541, 'volumefrom': 1170.85, 'volumeto': 1259491539.71}
{'time': 1525518000, 'close': 1074300, 'high': 1076954, 'low': 1070150, 'open': 1075997, 'volumefrom': 568.6, 'volumeto': 610397058.59}
{'time': 1525521600, 'close': 1077787, 'high': 1079900, 'low': 1074000, 'open': 1074300, 'volumefrom': 632.96, 'volumeto': 682058111.16}
{'time': 1525525200, 'close': 1074965, 'high': 1083035, 'low': 1074000, 'open': 1077787, 'volumefrom': 728.12, 'volumeto': 786321

# pybitflyer
__pybitflyer__ [[7](#ref07)]はbitFlyerのAPIをPythonで利用するために作製されたモジュール。   
最終更新は 2017/12 の様子。   
インストールはpipで行える。
```
pip install git+https://github.com/yagays/pybitflyer.git
pip install pybitflyer
```

In [7]:
!pip install pybitflyer

  Running setup.py bdist_wheel for pybitflyer ... - done
  Stored in directory: /content/.cache/pip/wheels/b1/f8/7d/de947eb15ac821af1baa340bc32fff796a951a36715dcbb9e2
Successfully built pybitflyer


In [0]:
import pybitflyer

api = pybitflyer.API()

Boardにはブローカーの売り・買い額（price）および所有量の一覧（size）が掲載されている。

In [38]:
board = api.board(product_code="BTC_JPY")
board["asks"][0]

{'price': 1078597.0, 'size': 0.2}

tickerにはリクエストした段階でのask, bid, size, volumeなどが含まれている。

In [26]:
ticker = api.ticker(product_code="BTC_JPY")
timestamp = ticker["timestamp"]
timestamp

'2018-05-05T17:29:20.66'

ticker中のtimestampはUNIX tmestampであり、それをdatetimeモジュールで変換するとUTCと同じものになる。

In [32]:
import datetime

datetimeFmt = "%Y-%m-%dT%H:%M:%S.%f"

print(datetime.datetime.strptime(timestamp, datetimeFmt).timestamp())
print(datetime.datetime.strptime(timestamp, datetimeFmt).\
      replace(tzinfo=datetime.timezone.utc).timestamp())

1525541360.66
1525541360.66


tickerのkeyを見ると、よく見られるbid と ask のほかにbid_sizeやask_sizeなるものがある。これらはブローカーが提供できるbidやaskの値の取引量を示す。

In [29]:
for key in ticker.keys():
  print(key)

product_code
timestamp
tick_id
best_bid
best_ask
best_bid_size
best_ask_size
total_bid_depth
total_ask_depth
ltp
volume
volume_by_product


ticker中のask, bidとboard中のasks, bidsの間には次の関係がある。   

* best_bid = max(bids)
* best_ask = min(asks)

ask = 買い、bid = 売り、であるから、高額で売り（best_bid）低額で買う（best_ask）、という理にかなっている。

In [35]:
ticker = api.ticker(product_code="BTC_JPY")
board = api.board(product_code="BTC_JPY")

ask_bid = [min([p["price"] for p in board["asks"]]), 
           max([p["price"] for p in board["asks"]]), 
           min([p["price"] for p in board["bids"]]), 
           max([p["price"] for p in board["bids"]])]
ticker, ask_bid

({'best_ask': 1078400.0,
  'best_ask_size': 0.06,
  'best_bid': 1078058.0,
  'best_bid_size': 0.01040104,
  'ltp': 1078058.0,
  'product_code': 'BTC_JPY',
  'tick_id': 8940715,
  'timestamp': '2018-05-05T17:43:34.937',
  'total_ask_depth': 2318.9526174,
  'total_bid_depth': 3214.74111504,
  'volume': 80901.21155476,
  'volume_by_product': 9576.34678477},
 [1078400.0, 2100000.0, 61895.0, 1078058.0])